In [1]:
from treble_tsdk.tsdk import TSDK, TSDKCredentials
from treble_tsdk import display_data as dd
from treble_tsdk import treble
import random
import json
from tqdm import tqdm
import glob
import math
from typing import List, Tuple
from collections import defaultdict

In [24]:
data_dir = "data"

rooms_data = []
for filename in tqdm(glob.glob(f"{data_dir}/**/*.json", recursive=True), desc="Loading room data"):
    with open(filename, "r") as f:
        data = json.load(f)
        room_name = filename.split("/")[-2]
        data["room_name"] = room_name
        rooms_data.append(data)

tsdk = TSDK(TSDKCredentials.from_file("./creds/tsdk.cred"))


Loading room data: 100%|██████████| 9/9 [00:00<00:00, 13534.86it/s]


== SDK package is up to date ==


In [25]:
project = tsdk.get_or_create_project("ertsi_9")

In [4]:
def sort_points_clockwise(points: List[List[float]]) -> List[List[float]]:
    # Compute centroid
    centroid_x = sum(x for x, y in points) / len(points)
    centroid_y = sum(y for x, y in points) / len(points)
    
    # Function to compute angle from centroid
    def angle_from_centroid(point):
        x, y = point
        return math.atan2(y - centroid_y, x - centroid_x)
    
    # Sort points by angle (clockwise)
    return sorted(points, key=angle_from_centroid, reverse=True)

In [26]:
room_definitions = []
for i, room_data in enumerate(rooms_data):
    edge_points_hexagon = sort_points_clockwise(room_data["room_verts"]) if not room_data["sorted"] else room_data["room_verts"]
    room_height = 3
    room = treble.GeometryDefinitionGenerator.create_polygon_room(
        points_xy=edge_points_hexagon, height_z=room_height, join_wall_layers=True
    )

    room_definitions.append(room)
room_definitions

[GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.]

In [27]:
generated_rooms = []

print("=== Populating rooms with geometry components ===")
for i, (room_def, room_data) in tqdm(enumerate(zip(room_definitions, rooms_data))):
    print(f"Populating room {room_data["room_name"]} with geometry components")
    room_def.clear_geometry_components()
    placements = []

    if room_data["chair_count"] > 0:
        placements += [
            treble.GeometryComponentPlacement(
                components=tsdk.geometry_component_library.query(group="chair"),
            preferred_count=room_data["chair_count"],
            rotation_settings=treble.ComponentAnglePool([0, 90, 180, 270]),
            min_dist_from_objects=0.5,
            min_dist_from_walls=0.5,
            )
        ]
    if room_data["desk_count"] > 0:
        placements += [
            treble.GeometryComponentPlacement(
               components=tsdk.geometry_component_library.query(group="desk"),
            preferred_count=room_data["desk_count"],
            rotation_settings=treble.ComponentAnglePool([0, 90, 180, 270]),
            min_dist_from_objects=0.5,
            min_dist_from_walls=0.5,
            )
        ]
        
    room_def.populate_with_geometry_components(
        components=placements,
        selection_algorithm=treble.ComponentSelectionAlgorithm.random,
    )

    try:
        model = project.add_model(f"room_{i}", room_def)
        generated_rooms.append(model)
    except Exception as e:
        print(f"Error adding model for room {room_data["room_name"]}: {e}")
        continue

=== Populating rooms with geometry components ===


0it [00:00, ?it/s]

Populating room jojo_3_cirlce with geometry components


1it [00:13, 13.97s/it]

Populating room jojo_4_unsorted with geometry components


...

7it [00:38,  4.95s/it]

Populating room jojo_2_sorted with geometry components


8it [00:46,  6.01s/it]

Populating room jojo_1_unsorted with geometry components


9it [00:49,  5.51s/it]


In [40]:
random_model = random.choice(generated_rooms)
random_model.plot()

In [41]:
dd.display(generated_rooms)

                                                 Models                                                 
┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Index ┃ Name   ┃ Id                                   ┃ Created At                   ┃ Is Watertight ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ 0     │ room_0 │ 54d2bc42-a1bf-4875-b517-2e9e6d50c7de │ 2025-05-26 14:30:39          │ True          │
│ 1     │ room_1 │ c224d105-d5fe-4149-bd2b-d686c3b90493 │ 2025-05-26 14:30:42          │ True          │
│ 2     │ room_2 │ 0d2089af-3797-4fe8-9af5-c260ab3b08f6 │ 2025-05-26T14:30:45.1366293Z │ False         │
│ 3     │ room_3 │ bc4c3445-a592-497e-892d-588f5baa8605 │ 2025-05-26 14:30:48          │ True          │
│ 4     │ room_4 │ 83281e49-0e0b-4a7d-9043-42d7310ff248 │ 2025-05-26 14:30:55          │ True          │
│ 5     │ room_5 │ 9cb38187-ee38-45ef-9ca9-f675fb236994 │ 2025-05-26 14:30:59          │ True          │
│ 6     │ room_6 │ ea83b519-454f-4384-8684-d7cb09c8e4ba │ 2025-05-26 14:31:04          │ True          │
│ 7     │ room_7 │ 29dde643-59ff-40ca-94bf-c81e3a76c12a │ 2025-05-26 14:31:12          │ True          │
│ 8     │ room_8 │ 64fc7026-66d3-49b7-821d-7ab5667c760c │ 2025-05-26 14:31:15          │ True          │
└───────┴────────┴──────────────────────────────────────┴──────────────────────────────┴───────────────┘

In [42]:
all_materials = tsdk.material_library.get()
database_materials = [
    material for material in all_materials if material["organizationId"] == None
]

all_material_assignments = []
for i, (model, room_data) in tqdm(enumerate(zip(generated_rooms, rooms_data))):
    layers = {
        "polygon_room_walls": "gypsum/plaster on solid backing",
        "polygon_room_floor": room_data["floor_material"],
        "polygon_room_ceiling": room_data["ceiling_material"],
        "Furniture/Desk": "wood",
        "Furniture/Chair": "upholstered concert chairs",
        "Furniture/Chair A": "upholstered concert chairs",
        "Furniture/Bar Stool": "upholstered concert chairs",
    }
    material_assignment = []

    for layer in model.layer_names:
        if layer in layers:
            search_string = layers[layer]
            matches = [
                m for m in database_materials if search_string.lower() in m.name.lower()
            ]
            if matches:
                material_assignment.append(
                    treble.MaterialAssignment(layer, random.choice(matches))
                )
    all_material_assignments.append(material_assignment)

dd.display(all_material_assignments[-1])

9it [00:00, 111.69it/s]


                                                Material assignment                                                
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer name           ┃ Material name                                                                            ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Furniture/Chair A    │ Upholstered concert chairs                                                               │
│ polygon_room_ceiling │ Striped panel, 12.0 mm stripes, at 80.0 mm intervals, 40 mm 81 kg/m3 absorber, 30.0 mm   │
│                      │ cavity                                                                                   │
│ Furniture/Desk       │ Wooden flooring                                                                          │
│ Furniture/Chair      │ Heavily upholstered concert chairs                                                       │
│ polygon_room_floor   │ Linoleum or vinyl on concrete                                                            │
│ polygon_room_walls   │ Gypsum/Plaster on solid backing                                                          │
└──────────────────────┴──────────────────────────────────────────────────────────────────────────────────────────┘

In [43]:
all_sources = []
all_source_positions = []
all_receivers = []
all_receiver_positions = []

pg = treble.PointsGenerator()

pos_ruleset = treble.PointRuleset(
    min_dist_from_surface=0.5,
    min_dist_from_other_points=2,
)

for room in tqdm(generated_rooms):
    pos = pg.generate_valid_points(
        model=room,
        max_count=2,	
        ruleset=pos_ruleset,
        z_range=(0.5, 1.5)
    )
    all_source_positions.append(pos[0])
    all_receiver_positions.append(pos[1])

    source = treble.Source.make_omni(
        position=pos[0],
        label=f"source_{room.name}",
    )
    receiver = treble.Receiver.make_mono(
        position=pos[1],
        label=f"receiver_{room.name}",
    )
    all_sources.append(source)
    all_receivers.append(receiver)


100%|██████████| 9/9 [00:08<00:00,  1.10it/s]


In [44]:
sim_type = treble.SimulationType.hybrid
crossover_frequency = 500

sim_defs = []

for i, (room, room_data) in tqdm(enumerate(zip(generated_rooms, rooms_data))):
    sim_def = treble.SimulationDefinition(
        name=f"simulation_{i}_1",  # unique name of the simulation
        simulation_type=sim_type,  # the type of simulation
        crossover_frequency=crossover_frequency,  # the frequency at which the simulation switches from wavebased to image source
        model=room,
        energy_decay_threshold=60,  # simulation termination criteria - the simulation stops running after -60 dB of energy decay
        receiver_list=[all_receivers[i]],
        source_list=[all_sources[i]],
        material_assignment=all_material_assignments[i],
    )
    sim_defs.append(sim_def)

sim_defs = project.add_simulations(sim_defs)

9it [00:00, 42414.31it/s]


In [45]:
all_sims = project.get_simulations()
dd.display(all_sims)

                                                      Simulations                                                      
┏━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━┳━━━━━┓
┃ Index ┃ Name           ┃ Created at       ┃ Id                                   ┃ Status  ┃ Freq. (Hz) ┃ Rcv ┃ Src ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━╇━━━━━┩
│ 0     │ simulation_0_1 │ 2025-05-26 14:32 │ c6aa07a0-5c44-41ba-85d6-e65219783e52 │ Created │ 500        │ 1   │ 1   │
│ 1     │ simulation_1_1 │ 2025-05-26 14:32 │ 7e010840-e5c5-467e-866f-3aa269423857 │ Created │ 500        │ 1   │ 1   │
│ 2     │ simulation_4_1 │ 2025-05-26 14:32 │ 177bd030-a0b6-43c5-90e0-db8db314f374 │ Created │ 500        │ 1   │ 1   │
│ 3     │ simulation_5_1 │ 2025-05-26 14:32 │ 43fc43a3-83b3-48f3-8f40-df846f48a3e7 │ Created │ 500        │ 1   │ 1   │
│ 4     │ simulation_6_1 │ 2025-05-26 14:32 │ dfc163f5-eb43-4963-b987-cdd41113e791 │ Created │ 500        │ 1   │ 1   │
│ 5     │ simulation_3_1 │ 2025-05-26 14:32 │ eb508da1-b21c-4529-91f3-f46605f189b8 │ Created │ 500        │ 1   │ 1   │
│ 6     │ simulation_2_1 │ 2025-05-26 14:32 │ f81dba61-8ccc-4ff4-80a5-8ac3ec81b669 │ Created │ 500        │ 1   │ 1   │
│ 7     │ simulation_7_1 │ 2025-05-26 14:32 │ 72442d5f-1861-4bb4-bc3a-398a62501b13 │ Created │ 500        │ 1   │ 1   │
│ 8     │ simulation_8_1 │ 2025-05-26 14:32 │ b3aff7ab-306b-4038-a691-68765f207bb3 │ Created │ 500        │ 1   │ 1   │
└───────┴────────────────┴──────────────────┴──────────────────────────────────────┴─────────┴────────────┴─────┴─────┘

In [52]:
random_sim = random.choice(all_sims)
random_sim.plot()

In [53]:
runtime_estimate = project.estimate()
dd.display(runtime_estimate)


                               Project 98e330e5-6acf-4e9f-a56a-bf22452916db estimate                               
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃                                                       ┃       Simulation ┃ Estimated runtime ┃   Estimated cost ┃
┃                                            Simulation ┃           status ┃           (hours) ┃         (tokens) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ simulation_0_1 (c6aa07a0-5c44-41ba-85d6-e65219783e52) │          Created │             0.863 │            0.884 │
│ simulation_1_1 (7e010840-e5c5-467e-866f-3aa269423857) │          Created │             0.085 │            0.108 │
│ simulation_4_1 (177bd030-a0b6-43c5-90e0-db8db314f374) │          Created │             1.536 │            1.607 │
│ simulation_5_1 (43fc43a3-83b3-48f3-8f40-df846f48a3e7) │          Created │             0.222 │            0.263 │
│ simulation_6_1 (dfc163f5-eb43-4963-b987-cdd41113e791) │          Created │             0.653 │            0.698 │
│ simulation_3_1 (eb508da1-b21c-4529-91f3-f46605f189b8) │          Created │             0.154 │            0.188 │
│ simulation_2_1 (f81dba61-8ccc-4ff4-80a5-8ac3ec81b669) │          Created │             0.084 │            0.105 │
│ simulation_7_1 (72442d5f-1861-4bb4-bc3a-398a62501b13) │          Created │             0.296 │            0.312 │
│ simulation_8_1 (b3aff7ab-306b-4038-a691-68765f207bb3) │          Created │              0.11 │            0.137 │
├───────────────────────────────────────────────────────┼──────────────────┼───────────────────┼──────────────────┤
│                                                 Total │                  │             1.536 │            4.302 │
└───────────────────────────────────────────────────────┴──────────────────┴───────────────────┴──────────────────┘

In [54]:
project.start_simulations()
project.as_live_progress()

                                          ertsi_9                                           
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Simulation Name ┃ Id                                   ┃ Status             ┃ Progress % ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ simulation_4_1  │ 177bd030-a0b6-43c5-90e0-db8db314f374 │ Completed          │ 100%       │
│ simulation_5_1  │ 43fc43a3-83b3-48f3-8f40-df846f48a3e7 │ Completed          │ 100%       │
│ simulation_7_1  │ 72442d5f-1861-4bb4-bc3a-398a62501b13 │ Completed          │ 100%       │
│ simulation_1_1  │ 7e010840-e5c5-467e-866f-3aa269423857 │ Completed          │ 100%       │
│ simulation_8_1  │ b3aff7ab-306b-4038-a691-68765f207bb3 │ Completed          │ 100%       │
│ simulation_0_1  │ c6aa07a0-5c44-41ba-85d6-e65219783e52 │ Completed          │ 100%       │
│ simulation_6_1  │ dfc163f5-eb43-4963-b987-cdd41113e791 │ InsufficientTokens │ 0%         │
│ simulation_3_1  │ eb508da1-b21c-4529-91f3-f46605f189b8 │ InsufficientTokens │ 0%         │
│ simulation_2_1  │ f81dba61-8ccc-4ff4-80a5-8ac3ec81b669 │ InsufficientTokens │ 0%         │
├─────────────────┼──────────────────────────────────────┼────────────────────┼────────────┤
│                 │                                      │ Created            │ 66.67%     │
└─────────────────┴──────────────────────────────────────┴────────────────────┴────────────┘

Interrupted

In [55]:
for sim, room in zip(all_sims, rooms_data):
    room_name = room["room_name"]
    
    try:
        r = sim.download_results(f'data/{room_name}')
    except ValueError as e:
        print(f"Room {room_name} already exists, skipping download.")
        r = None

/home/leo/projects/repos/ertsi/generation/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

Download simulation c6aa07a0-5c44-41ba-85d6-e65219783e52:   0%|          | 0/2 tasks

Download simulation c6aa07a0-5c44-41ba-85d6-e65219783e52: 100%|██████████| 2/2 tasks
Download simulation 7e010840-e5c5-467e-866f-3aa269423857: 100%|██████████| 2/2 tasks
Download simulation 177bd030-a0b6-43c5-90e0-db8db314f374: 100%|██████████| 2/2 tasks
Download simulation 43fc43a3-83b3-48f3-8f40-df846f48a3e7: 100%|██████████| 2/2 tasks
Download simulation 72442d5f-1861-4bb4-bc3a-398a62501b13: 100%|██████████| 2/2 tasks
Download simulation b3aff7ab-306b-4038-a691-68765f207bb3: 100%|██████████| 2/2 tasks


In [59]:
for simulation, room in zip(all_sims, rooms_data):
    try:
        results = simulation.get_results_object(f'data/{room["room_name"]}')
        if not results:
            print(f"No results found for room {room['room_name']}")
            continue
        mono_ir = results.get_mono_ir(source=simulation.sources[0], receiver=simulation.receivers[0])
        mono_ir.write_to_wav(path_to_file=f"data/{room['room_name']}/ir.wav")
    except KeyError as e:
        print(f"Results not found for room {room['room_name']}: {e}")


Simulation not completed


No results found for room jojo_8_unsorted


Simulation not completed
Simulation not completed


No results found for room jojo_7_sorted
No results found for room jojo_6_unsorted
